In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


/Users/pasqualeiuliano/Library/Python/3.6/lib/python/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/pasqualeiuliano/Library/Python/3.6/lib/python/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/pasqualeiuliano/Library/Python/3.6/lib/python/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/pasqualeiuliano/Library/Pyt

In [2]:
tf.enable_eager_execution()
csv_path = "/Users/pasqualeiuliano/PycharmProjects/Practice_ML/EmotionPractice.csv"
dataframe = pd.read_csv(csv_path)
dataframe.head()

,sysBP,diaBP,heartRate,emotion
0,106.0,70.0,80,0
1,121.0,81.0,95,0
2,127.5,80.0,75,0
3,150.0,95.0,65,1
4,130.0,84.0,85,0


In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

np.set_printoptions(precision=3, suppress=True)

95 train examples
24 validation examples
30 test examples


In [4]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('emotion')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [5]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of targets:', label_batch)

Every feature: ['sysBP', 'diaBP', 'heartRate']
A batch of targets: tf.Tensor([0 0 0 1 0], shape=(5,), dtype=int32)


In [6]:
example_batch = next(iter(train_ds))[0]

In [7]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())
  
  

In [8]:
feature_columns = []

# numeric cols
for header in ['sysBP', 'diaBP', 'heartRate']:
  feature_columns.append(feature_column.numeric_column(header))

In [9]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [10]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [11]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
3/3 [==============================] - 2s 671ms/step - loss: 2.5700 - acc: 0.4407 - val_loss: 1.5229 - val_acc: 0.6250
Epoch 2/5
3/3 [==============================] - 0s 6ms/step - loss: 1.4254 - acc: 0.6521 - val_loss: 1.0136 - val_acc: 0.5417
Epoch 3/5
3/3 [==============================] - 0s 6ms/step - loss: 1.1346 - acc: 0.5128 - val_loss: 0.8656 - val_acc: 0.5000
Epoch 4/5
3/3 [==============================] - 0s 8ms/step - loss: 0.6061 - acc: 0.6534 - val_loss: 1.2536 - val_acc: 0.5833
Epoch 5/5
3/3 [==============================] - 0s 5ms/step - loss: 0.7823 - acc: 0.6991 - val_loss: 0.8022 - val_acc: 0.6250


W1208 21:11:40.584090 140734922603968 deprecation.py:323] From /Users/pasqualeiuliano/Library/Python/3.6/lib/python/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 53ms/step - loss: 0.5835 - acc: 0.8333
Accuracy 0.8333333


In [13]:

predictions = model.predict(test_ds)


In [14]:
for prediction, emotion in zip(predictions[:10], list(test_ds)[0][1][:10]):
  print(prediction[0])
  if prediction[0] > 0.5:
      print("Model Prediction: Happy")
  else:
      print("Model Prediction: Sad")
      
  if bool(emotion):
      print("Actual: 1 / Happy \n")
  else:
      print("Actual: 0 / Sad \n")

0.8918048
Model Prediction: Happy
Actual: 1 / Happy 

0.031570733
Model Prediction: Sad
Actual: 0 / Sad 

0.93347275
Model Prediction: Happy
Actual: 1 / Happy 

0.73356044
Model Prediction: Happy
Actual: 1 / Happy 

0.37868187
Model Prediction: Sad
Actual: 0 / Sad 

0.48846167
Model Prediction: Sad
Actual: 0 / Sad 

0.61361563
Model Prediction: Happy
Actual: 1 / Happy 

0.61887515
Model Prediction: Happy
Actual: 0 / Sad 

0.97319007
Model Prediction: Happy
Actual: 1 / Happy 

0.9141314
Model Prediction: Happy
Actual: 0 / Sad 



In [15]:
 
# Save the model
model.save('database.lite')